# ThorViz Notebook


In [24]:
from typing import Dict
import numpy as np
import requests
import pandas as pd

# Resource: https://testnet.midgard.thorchain.info/v2/doc#operation/GetPools


In [21]:
my_request = 'https://midgard.thorchain.info/v2/pools'
    
r = requests.get(my_request)
response_dict = r.json()
response_dict.keys()

AttributeError: 'list' object has no attribute 'keys'

In [30]:
pool_df = pd.DataFrame(response_dict)

In [31]:
pool_df.head(2)

,asset,assetDepth,assetPrice,assetPriceUSD,poolAPY,runeDepth,status,units,volume24h
0,BCH.BCH,15110496222,64.6317940706739,832.722190908846,0.2690831805870779,976618480126,available,944477365474,10132838293
1,ETH.USDT-0XDAC17F958D2EE523A2206206994597C13D8...,87973195818300,0.07700358576299253,0.9921215334712705,0.12291418250868902,6774251529039,available,8995653916193,1154156467253


In [144]:
pool_df['assetDepth'] = pool_df['assetDepth'].astype(float) / 1e8 


In [147]:
pool_df['assetPriceUSD'] = pool_df['assetPriceUSD'].astype(float)

In [150]:
pool_df[pool_df['asset'] == 'BTC.BTC']['assetDepth'] #.astype(float) / 1e8

13    33.253712
Name: assetDepth, dtype: float64

In [152]:
pool_df[pool_df['asset'] == 'BTC.BTC']['tv_usd'].round(2)

13    1669700.08
Name: tv_usd, dtype: float64

In [146]:
pool_df[pool_df['asset'] == 'BNB.BNB']['assetDepth']# .astype(float) / 1e8

8    1228.575408
Name: assetDepth, dtype: float64

In [148]:
# Calculate total value Locked for NON rune 

# Locked Value
pool_df['tv_usd'] = pool_df['assetPriceUSD'] * pool_df['assetDepth']


In [ ]:
pool_df['runeDepth'] = pool_df['runeDepth'].astype(float) / 1e8
pool_df['rune_vt_usd'] = pool_df['runeDepth'] * 

In [149]:
# Sum of locked value
pool_df['tv_usd'].sum()

6505367.329979617

In [35]:
pool_df['runeDepth'].astype(float).sum() / 100000000

504914.56360307

In [51]:
pool_df['adj_runeDepth'] = pool_df['runeDepth'].apply(lambda x: int(x) / 100_000_000)

In [53]:
pool_df['adj_runeDepth'].sum()

504914.56360307004

Note: can also get the value from: https://midgard.thorchain.info/v2/stats
`runeDepth column` 

In [54]:
50518348515320 

505183.4851532

In [52]:
pool_df[pool_df['asset'] == 'BNB.BNB']['adj_runeDepth'].values #.value / 1000000

array([49425.94532057])

# Now lets get Bonded Rune

In [60]:
network_data_request = 'https://midgard.thorchain.info/v2/network'
    
rn = requests.get(network_data_request)
net_response_dict = rn.json()

In [65]:
net_response_dict['bondMetrics']

{'averageActiveBond': '25693937378778',
 'averageStandbyBond': '10000407000000',
 'maximumActiveBond': '29978837503545',
 'maximumStandbyBond': '30000392000001',
 'medianActiveBond': '29972728553814',
 'medianStandbyBond': '490000002',
 'minimumActiveBond': '23408905603',
 'minimumStandbyBond': '390000000',
 'totalActiveBond': '179857561651449',
 'totalStandbyBond': '60002442000004'}

# SLAW method

In [24]:
import ftx

In [27]:


def get_market_price() -> float:
    
    ftx_client = ftx.FtxClient()
    
    result = ftx_client.get_market('RUNE/USD')
    
    market_price = result['price']
    
    return market_price
    

In [28]:
get_market_price()

17.8605

In [44]:
!pip list | grep c

ftx                            1.0.2


In [40]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [41]:
import pandas as pd
import ftx
import numpy 
import requests


def get_market_price() -> float:
    
    ftx_client = ftx.FtxClient()
    
    result = ftx_client.get_market('RUNE/USD')
    
    market_price = result['price']
    
    return market_price


def get_rune_stats() -> pd.DataFrame:
    
    '''
    Slaw's method.
    '''
    
    market_price = get_market_price()
    
    # MCCN
    mccn = requests.get('https://midgard.thorchain.info/v2/network')
    mccn_dict = mccn.json()
    
    mccn_total_pooled_rune = float(mccn_dict['totalPooledRune']) / 1e7
    mccn_total_active_bond = float(mccn_dict['bondMetrics']['totalActiveBond']) / 1e7 
    
    # ---
    
    # SCCN
    sccn = requests.get('http://thorb.sccn.nexain.com:8080/v1/network')
    sccn_dict = sccn.json()
    
    sccn_total_staked_rune = float(sccn_dict['totalStaked']) / 1e7
    sccn_total_active_bond = float(sccn_dict['bondMetrics']['totalActiveBond']) / 1e7 
    
    # calculations
    
    rune_in_lp_count = mccn_total_pooled_rune + sccn_total_staked_rune
    rune_bonded_count = mccn_total_active_bond + sccn_total_active_bond
    
    total_in_network_count = rune_in_lp_count + rune_bonded_count
    
    deterministic_value = rune_in_lp_count * market_price * 3 # In USD
    
    determined_price = deterministic_value / total_in_network_count # In USD
    
    speculation = market_price - determined_price # USD
    
    speculation_pct = speculation / market_price
    
    # Collect Results
    result_dict = {
        'Rune_in_LP_count': rune_in_lp_count,
        'Rune_bonded_count': rune_bonded_count,
        'total_in_network_count': total_in_network_count,
        'deterministic_value_usd': deterministic_value,
        'determined_price': determined_price,
        'market_price_usd': market_price,
        'speculation_premium_usd': speculation,
        'speculation_pct_of_market': speculation_pct,
    }
    
    return pd.DataFrame(result_dict, index=[0]).T.round(2)

In [43]:
get_rune_stats()

,0
Rune_in_LP_count,71657739.39000
Rune_bonded_count,261665341.15000
total_in_network_count,333323080.54000
deterministic_value_usd,3834047346.03000
determined_price,11.50000
market_price_usd,17.84000
speculation_premium_usd,6.33000
speculation_pct_of_market,0.36000


In [ ]:
RUNE_in_LP_count = totalStaked_SCCN + totalPooleRune_MCCN
RUNE_bonded_count = totalActiveBond_SCCN + totalActiveBond_MCCN

total_in_network_count = RUNE_in_LP_count + RUNE_bonded_count

deterministic_value = RUNE_in_LP_count * market_price * 3
determined_price = deterministic_value / total_in_network_count
speculation = market_price - determined_price

# In-Network Rune Calculation
For the in-network rune calc we need the following values: 
1. `runeDepth`, depth of rune in LP. source: https://midgard.thorchain.info/v2/stats
2. `totalActiveBond`, source: https://midgard.thorchain.info/v2/network
3. `totalStandbyBond`, source: same as (2)
4. `totalReserve`, source: same as (2) 
5. `totalPooledRune`, source: same as (2)  <-  Ask if this is same as `runeDepth`?

In [160]:
my_request = 'https://midgard.thorchain.info/v2/pools'
    
r = requests.get(my_request)
response_dict = r.json()
#    response_dict.keys()

In [162]:
pd.DataFrame(response_dict)

,asset,assetDepth,assetPrice,assetPriceUSD,poolAPY,runeDepth,status,units,volume24h
0,BCH.BCH,15111866639,64.62029491352236,778.9774124507321,0.2392274556685896,976533278906,available,944477365474,288263814
1,ETH.USDT-0XDAC17F958D2EE523A2206206994597C13D8...,84442557269000,0.08372040602783039,1.0092232686984675,0.16075945826115778,7069565180589,available,8995653916193,1344310318384
2,ETH.AAVE-0X7FC66500C84A76AD7E9C93437BFC5AC33E2...,14899683384,30.253844800357403,364.70062185273054,0,450772708674,staged,468222550349,0
3,ETH.LINK-0X514910771AF9CA656AF840DFF83E8264ECF...,33000000000,2.757575757575758,33.24171192885913,0,91000000000,staged,101000000000,0
4,ETH.TVK-0XD084B83C305DAFD76AE3E1B4E1F1FE2ECCCB...,5500000000000,0.03854848484854546,0.46468990928316684,0,212016666667,staged,222016666666,0
5,ETH.YFI-0X0BC529C00C6401AEF6D220BE8C6EA1667F6A...,185744648,3422.8293278199867,41261.13532310333,0,635772228660,staged,655059459431,0
6,BNB.BTCB-1DE,660828118,4267.540741538181,51443.86680407811,0.11845309985985786,2820110916719,available,2816600000000,789711432474
7,ETH.SUSHI-0X6B3595068778DD592E39A122F4F5A5CF09...,5169297807562,0.914402812952716,11.02283946745843,0.03659203956182022,4726820456225,available,5766694937746,53207371260
8,BNB.BNB,119623161394,42.583455019309504,513.3302107108215,0.3236829963168386,5093967512489,available,4921048281472,3106971217328
9,ETH.CREAM-0X2BA592F78DB6436527729929AAF6C90849...,10000000000,9.49,114.39897719517377,0,94900000000,staged,105900000000,0


In [ ]:
def get_multichain_pool_data():
    
    my_request = 'https://midgard.thorchain.info/v2/pools'
    
    r = requests.get(my_request)
    response_dict = r.json()
    response_dict.keys()
    

In [159]:
def get_singlechain_stats():
    
    '''
    Returns Datafraome containing global stats for all books and all transaction from BEPSwap.
    
    Documentation: https://testnet.midgard.thorchain.info/v2/doc#operation/GetStats
    '''
    
    _url = "https://midgard.thorchain.info/v2/stats"
    rn = requests.get(_url)
    response_dict = rn.json()
    
    return pd.DataFrame([response_dict]).astype(float)

In [3]:
import requests
import pandas as pd

def get_non_rune_TVL():
    
    my_request = 'https://midgard.thorchain.info/v2/pools'
    
    r = requests.get(my_request)
    response_dict = r.json()
    
    
    return pd.DataFrame(response_dict)

    

In [4]:
df = get_non_rune_TVL()

In [10]:
df.to_csv('pool-data-2021-05-05.csv')

In [5]:
df.head(2)

,asset,assetDepth,assetPrice,assetPriceUSD,poolAPY,runeDepth,status,units,volume24h
0,BCH.BCH,42239303050,73.73112763637326,1355.3701716574021,0.13187937916761494,3114351444451,available,2535394661449,939793610465
1,ETH.USDT-0XDAC17F958D2EE523A2206206994597C13D8...,215238111837900,0.05439925503614397,0.9999999999999999,0.05000385758213488,11708792939368,available,18410844581074,1580401470322


In [6]:
def calculate_non_rune_TVL(df) -> float:
    '''
    Calulates total non Rune value locked in the Network by summing
    all Pool USD value.
    '''
    df = df.copy()
    
    df['total_value_USD'] = df['assetPriceUSD'].astype(float) * (df['assetDepth'].astype(float) / 1e7 )
    
    total_non_rune_tvl = df['total_value_USD'].sum()
    
    return total_non_rune_tvl


In [7]:
tvl = calculate_non_rune_TVL(df)

In [8]:
tvl

261813078.07097054

In [157]:
v2_stats_df = get_v2_stats()

In [158]:
v2_stats_df

,addLiquidityCount,addLiquidityVolume,dailyActiveUsers,impermanentLossProtectionPaid,monthlyActiveUsers,runeDepth,runePriceUSD,swapCount,swapCount24h,swapCount30d,swapVolume,switchedRune,toAssetCount,toRuneCount,uniqueSwapperCount,withdrawCount,withdrawVolume
0,1094.0,1.266926e+14,209.0,6.029487e+09,2002.0,5.231769e+13,12.021922,21209.0,1537.0,21209.0,4.819078e+14,7.330460e+14,8345.0,12864.0,2002.0,270.0,1.564070e+13


In [ ]:
v2_stats_df['runeDepth'].astype(float)

In [113]:
v2_stats_df.columns.tolist()

['addLiquidityCount',
 'addLiquidityVolume',
 'dailyActiveUsers',
 'impermanentLossProtectionPaid',
 'monthlyActiveUsers',
 'runeDepth',
 'runePriceUSD',
 'swapCount',
 'swapCount24h',
 'swapCount30d',
 'swapVolume',
 'switchedRune',
 'toAssetCount',
 'toRuneCount',
 'uniqueSwapperCount',
 'withdrawCount',
 'withdrawVolume']

In [105]:
def get_NetworkRuneMetrics_stats():
    
    network_data_request = 'https://midgard.thorchain.info/v2/network'
    rn = requests.get(network_data_request)
    net_response_dict = rn.json()
    
    _df = pd.DataFrame([net_response_dict['bondMetrics']]).astype(float)
    
    _df['totalPooledRune'] = float(net_response_dict['totalPooledRune'])
    _df['totalReserve'] = float(net_response_dict['totalReserve'])
    
    return _df
    

In [106]:
networkrune_df = get_NetworkRuneMetrics_stats()

In [111]:
networkrune_df.to_dict()

{'averageActiveBond': {0: 25671897062889.0},
 'averageStandbyBond': {0: 10000406666667.0},
 'maximumActiveBond': {0: 29953060050449.0},
 'maximumStandbyBond': {0: 30000392000001.0},
 'medianActiveBond': {0: 29947079283072.0},
 'medianStandbyBond': {0: 488000002.0},
 'minimumActiveBond': {0: 23388752868.0},
 'minimumStandbyBond': {0: 390000000.0},
 'totalActiveBond': {0: 179703279440225.0},
 'totalStandbyBond': {0: 60002440000004.0},
 'totalPooledRune': {0: 50338750079290.0},
 'totalReserve': {0: 485094506917.0}}

# Calculate In-Network Rune

In [126]:
def calc_in_network_rune(network_rune_df: pd.DataFrame, stats_df: pd.DataFrame):
    
    assert network_rune_df.shape[0] == 1, "`network_rune_df` is wrong shape, should be single row"
    # TODO add assert for stats_df
    
    total_bond = network_rune_df['totalActiveBond'][0] + network_rune_df['totalStandbyBond'][0]
    
    # TODO: Verify that `runeDepth`
    total_pooled = network_rune_df['totalPooledRune'][0] + stats_df['runeDepth'][0]
    
    print((network_rune_df['totalPooledRune'][0] - stats_df['runeDepth'][0])/ 1e7)
    
    total_reserve = network_rune_df['totalReserve'][0]
    
    return np.round((total_bond + total_pooled + total_reserve) / 1e7, 2)

In [127]:
calc_in_network_rune(network_rune_df=networkrune_df, stats_df=v2_stats_df)

311.8446297


34086519.57

In [100]:
bond_df['totalActiveBond'].values + bond_df['totalStandbyBond'].values 

array([2.39705719e+14])

In [66]:
#bond_df = pd.DataFrame([net_response_dict['bondMetrics']])

In [79]:
#bond_df = bond_df.astype(int)

In [80]:
#total_bond_rune = bond_df['totalActiveBond'] + bond_df['totalStandbyBond']

In [82]:
total_bond_rune[0] / 100_000_000

2398600.03651453

In [73]:
17 + 6 + 

23

In [ ]:
"totalPooledRune": "50518030721572",
  "totalReserve": "484395137229"

In [83]:
50518030721572 + 484395137229 + 239860003651453

290862429510254

In [86]:
34 - (29 + 4.8) # Total Reserve, total pooled + total bonded (both active and standby)

0.20000000000000284

In [89]:
34 - 29

5

In [ ]:
"totalPooledRune": "50518030721572",
"totalReserve": "484395137229",
"totalActiveBond": "179864730344791",
"totalStandbyBond": "60002442000004",

In [88]:
(50518030721572
+ 484395137229
+ 179864730344791
+ 60002442000004)

290869598203596

In [90]:
# from: https://midgard.thorchain.info/v2/stats
50449426289833  + 290869598203596

341319024493429

In [77]:
 (   390000000
    + 392000000
    + 486000000
    + 490000002
    + 30000292000001
    + 30000392000001) / 100_000_000

600024.42000004

In [78]:
60002442000004 / 100_000_000


600024.42000004

In [ ]:
"totalActiveBond": "179864730344791",
    "totalStandbyBond": "60002442000004"

In [36]:
pool_df[pool_df['asset']]

,asset,assetDepth,assetPrice,assetPriceUSD,poolAPY,runeDepth,status,units,volume24h
0,BCH.BCH,15110496222,64.6317940706739,832.722190908846,0.2690831805870779,976618480126,available,944477365474,10132838293
1,ETH.USDT-0XDAC17F958D2EE523A2206206994597C13D8...,87973195818300,0.07700358576299253,0.9921215334712705,0.12291418250868902,6774251529039,available,8995653916193,1154156467253


In [ ]:
def get_data():
    
    
    my_request = 'https://midgard.thorchain.info/v2/pools'
    
    r = requests.get(my_request)
    response_dict = r.json()
    response_dict.keys()

In [ ]:
So we need to get all the pool data for pool rune
and we need to get bonded rune and sum

In [10]:
value_dict = {
    'non_rune_TVL': 91, # in Million USD
    'in_network_rune': 33.6, # in Million USD 
    'market_price': 13.45,
    
}

In [133]:
def new_det_price_calc(non_rune_tvl, circ_supply):
    
    '''
    source: https://thorchain.org/rune#what-influences-it
    '''
    
    det_price = (3 * non_rune_tvl) / circ_supply
    
    return det_price

In [134]:
new_det_price_calc(non_rune_tvl=13_000_000, circ_supply=235_000_000)

0.16595744680851063

In [19]:
def calculate_deterministic_price(value_dict: Dict[str, float]):
  
 NEED TO REVISE THIS!!!!!
#     det_value = value_dict.get('non_rune_TVL') * 3
    
#     # Deterministic price per Rune, `baseline` from @Slawteh
#     det_price = det_value / value_dict.get('in_network_rune')
    
#     # Speculation Premium 
#     spec_prem = value_dict.get('market_price') - det_price
    

#     print(f"Deterministic Price, USD: ${np.round(det_price ,2)}")
#     print(f"Speculative Premium, USD: ${np.round(spec_prem ,2)}")
#     #return 
    

In [12]:
calculate_deterministic_price(value_dict)

Rune Cap Elasticity, USD: $-0.23
Deterministic Price, USD: $8.12
Speculative Premium, USD: $5.32


In [17]:
def calculate_rune_elasticity(value_dict: Dict[str, float], det_price: float):
    
    # Rune Cap, Deterministic Price elasticity
    new_det_value = (1 + value_dict.get('non_rune_TVL')) * 3
    
    new_det_price = new_det_value / (1 + value_dict.get('in_network_rune'))
    # increase of 1M rune to det price
    #inc_det_price = det_value / ( 1 + value_dict.get('in_network_rune'))
    # Rune Cap Elasticity, +1M Rune -> +x price increase in Rune price.
    cap_elasticity = new_det_price - det_price 
    
    
    print(new_det_price)
    print(f"Rune Cap Elasticity, USD: ${np.round(cap_elasticity ,2)}")
    


In [18]:
calculate_rune_elasticity(value_dict, det_price=8.12)

7.976878612716763
Rune Cap Elasticity, USD: $-0.14
